# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [1]:
import numpy as np
import h5py
import os
import sys
import json
import requests
from IPython.display import IFrame

In [2]:
%load_ext autoreload
%autoreload 2



sys.path.insert(0, '/Users/ageller/VISUALIZATIONS/Firefly')
sys.path.insert(0,'/Users/agurvich/research/repos/Firefly/src')
from Firefly.data_reader import SimpleReader
from Firefly.server import spawnFireflyServer,killAllFireflyServers

## Start the server

In [3]:
## spawnFireflyServer takes same *positional* arguments as startFireflyServer
process = spawnFireflyServer()

## Show Firefly in an IFrame

In [4]:
url = "http://localhost:5000"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [5]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("foo.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to foo.hdf5")

np.savetxt('foo.csv',coords)
print("output to foo.csv")

if not os.path.isdir('foo_csv'):
    os.mkdir('foo_csv')
    #os.rename('foo.csv','../static/data/foo_csv/foo.csv')

output to foo.hdf5
output to foo.csv


## Send this data to the Flask app

In [8]:
!pwd

/Users/agurvich/research/repos/my_Firefly/src/Firefly/ntbks


In [19]:
## test out the csv
fdir = '/Users/agurvich/research/repos/Firefly/static/data/foo_csv'
fdir = '/Users/ageller/VISUALIZATIONS/Firefly/static/data/foo_csv'

simple = SimpleReader(
    "foo.csv",
    write_jsons_to_disk=False,
    extension='.csv',
    JSONdir='foo_csv')
#simple = SimpleReader(fdir,write_jsons_to_disk=False,extension='.csv')
#simple.sendDataViaFlask()

Opening 1 files and 1 particle types...
JSONdir: /Users/agurvich/foo_csv -- is not a sub-directory of Firefly/static/data. 
This may produce confusing or inoperable results. As such, we will create a symlink for you when you  dumpToJSON.


In [21]:
simple.copyFireflySourceToTarget(init_gh_pages=False)

['/Users/agurvich/my_Firefly']

In [8]:
import shutil

In [10]:
shutil.rmtree('/Users/agurvich/miniconda3/envs/firefly/lib/python3.9/site-packages/Firefly-2.0.0-py3.9.egg/Firefly/static/data/foo_csv')

In [11]:
simple.dumpToJSON()

outputting: 
ParticleGroup: foo
Contains 7999 particles (7999 after decimation) and 0 tracked fields


In [ ]:
## test out the hdf5
fdir = '/Users/ageller/VISUALIZATIONS/Firefly/static/data/foo_hdf5'
#simple = SimpleReader("foo.hdf5",write_jsons_to_disk=False,extension='.hdf5')
simple = SimpleReader(fdir,write_jsons_to_disk=False,extension='.hdf5')
simple.sendDataViaFlask()

In [ ]:
## alternatively, make the request yourself (literally just copying the code from sendDataViaFlask())
port = 5000
print('sending to Firefly', sys.getsizeof(simple.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=simple.JSON)

In [ ]:
#test with FIRE data
fdir = '/Users/ageller/VISUALIZATIONS/FIREdata/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=100)
reader.sendDataViaFlask()

*To kill the Firefly server.*

In [13]:
killAllFireflyServers(pid=process.pid)